# **Live Microfluidics Predictions Notebook**
---

## Introduction

<font size = 4> The "Live Microfluidics Predictions" Jupyter Notebook is part of a deep learning-based image analysis pipeline designed for processing videos of circulating cells attaching to endothelial monolayers. It incorporates state-of-the-art deep learning models from the [ZeroCostDL4Mic](https://github.com/HenriquesLab/ZeroCostDL4Mic) project, facilitating the segmentation of circulating cells, identification of endothelial cell nuclei, and junctions from microscopy videos.

## Pipeline Overview

<font size = 4> The notebook is a component of a comprehensive pipeline that processes raw microscopy videos into analyzable data, offering insights into cell attachment dynamics. The steps involved include video pre-processing, cell segmentation, frame selection, artificial labeling, image summation, nuclei segmentation, endothelial cell segmentation, and outline generation.

### Key Features

- **Cell Segmentation:** Utilizes StarDist models for segmenting circulating cells.
- **Frame Selection:** Identifies frames with optimal contrast for further analysis.
- **Artificial Labeling:** Applies pix2pix models for simulating staining processes.
- **Image Summation:** Aggregates predictions to generate a comprehensive image per video segment.
- **Nuclei & Endothelial Cell Segmentation:** Employs StarDist and Cellpose models for detailed segmentation.


## Contributing

<font size = 4> We encourage contributions that enhance the functionality or extend the capabilities of this pipeline.

## Support

<font size = 4> For assistance or to address queries, please open an issue in the repository. Our team is committed to providing support and ensuring users can leverage this pipeline effectively for their research needs.

<font size = 4>Notebook created by [Guillaume Jacquemet](https://cellmig.org/)






# **1. Initialise the Colab session**




---







## **1.1. Check for GPU access**
---

<font size = 4> By default, the session should be using Python 3 and GPU acceleration, but it is possible to ensure that these are set properly by doing the following:

<font size = 4>Go to **Runtime -> Change the Runtime type**

<font size = 4>**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

<font size = 4>**Accelator: GPU** *(Graphics processing unit)*


In [ ]:
#@markdown ##Run this cell to check if you have GPU access

import tensorflow as tf
if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.')
  print('Did you change your runtime ?')
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')

else:
  print('You have GPU access')
  !nvidia-smi

## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab


# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

##**1.3. Test OpenCL status**
---


In [ ]:
#@markdown ##test pyclesperanto

!pip install -q pyclesperanto-prototype


#For pyclesperanto

import pyclesperanto_prototype as cle

gpu_devices = cle.available_device_names(dev_type="gpu")
print("Available GPU OpenCL devices:" + str(gpu_devices))
cle.select_device(gpu_devices[0])
print("Using OpenCL device " + cle.get_device().name)

In [ ]:
#@markdown ##Fix OpenCL if needed


!sudo apt-get update -qq
!sudo apt-get purge -qq *nvidia* -y
!sudo DEBIAN_FRONTEND=noninteractive apt-get install -qq nvidia-driver-530 -y
exit(0)


In [ ]:
#@markdown ##test pyclesperanto again after install


import pyclesperanto_prototype as cle

gpu_devices = cle.available_device_names(dev_type="gpu")
print("Available GPU OpenCL devices:" + str(gpu_devices))
cle.select_device(gpu_devices[0])
print("Using OpenCL device " + cle.get_device().name)

## **1.4. Install StarDist, Cellpose, Pix2pix and dependencies**
---


In [ ]:
#@markdown ## StarDist, Cellpose, Pix2pix, and Dependencies Installation

# Install necessary packages
!pip install -q tifffile folium==0.2.1 imgaug==0.2.5 PTable zarr imagecodecs wget memory_profiler fpdf2 csbdeep stardist gputools cellpose

# Optional: Uncomment the following lines if there's a need to uninstall specific packages
# !pip uninstall -y yellowbrick

# Load memory_profiler for monitoring memory usage
%load_ext memory_profiler

# Clone the repository for pix2pix models
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

# Change directory to the cloned repository and install requirements
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')
!pip install -r requirements.txt

# Optional: Uncomment the following line to force a session restart (useful when required by certain installations)
# exit(0)


## **1.5. Load the dependencies and functions**
---


In [ ]:
#@markdown ##Load the dependencies and functions

# Enabling inline plotting and memory profiling
%matplotlib inline
%load_ext memory_profiler

from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
np.random.seed(42)

import tensorflow
print(tensorflow.__version__)
print("Tensorflow enabled.")

import imagecodecs

# ------- Variable specific to Stardist -------
from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available, relabel_image_stardist, random_label_cmap,  relabel_image_stardist, _draw_polygons, export_imagej_rois
from stardist.models import Config2D, StarDist2D, StarDistData2D # import objects
from stardist.matching import matching_dataset
from csbdeep.utils import Path, normalize, download_and_extract_zip_file, plot_history # for loss plot
from csbdeep.io import save_tiff_imagej_compatible
%matplotlib inline

lbl_cmap = random_label_cmap()

import os.path
from PIL import Image
import zarr
from zipfile import ZIP_DEFLATED
from csbdeep.data import Normalizer, normalize_mi_ma


class MyNormalizer(Normalizer):
    def __init__(self, mi, ma):
            self.mi, self.ma = mi, ma
    def before(self, x, axes):
        return normalize_mi_ma(x, self.mi, self.ma, dtype=np.float32)
    def after(*args, **kwargs):
        assert False
    @property
    def do_after(self):
        return False

# ------- Common variable to all ZeroCostDL4Mic notebooks -------
import numpy as np
from matplotlib import pyplot as plt
import urllib
import shutil
from tifffile import imread, imsave
import time
import wget
from pathlib import Path
import pandas as pd
import csv
from skimage import io
from sklearn.linear_model import LinearRegression
from skimage.util import img_as_uint
import matplotlib as mpl
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr
from astropy.visualization import simple_norm
from skimage import img_as_float32, img_as_ubyte, img_as_float
from skimage.util import img_as_ubyte
from tqdm import tqdm
from fpdf import FPDF, HTMLMixin
from datetime import datetime
from pip._internal.operations.freeze import freeze
import subprocess

# For sliders and dropdown menu and progress bar

import ipywidgets as widgets

# Colors for the warning messages
class bcolors:
  WARNING = '\033[31m'
W  = '\033[0m'  # white (normal)
R  = '\033[31m' # red

import imageio
from skimage import data
from skimage import exposure
from skimage.exposure import match_histograms
import os.path


# ------- Common variable to all ZeroCostDL4Mic notebooks -------
import numpy as np
from matplotlib import pyplot as plt
import urllib
import os, random
import shutil
import zipfile
from tifffile import imread, imsave
import sys
from glob import glob
from scipy import signal
from scipy import ndimage
from skimage import io
from sklearn.linear_model import LinearRegression
from skimage.util import img_as_uint
import matplotlib as mpl
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr
from astropy.visualization import simple_norm
from skimage import img_as_float32
from skimage.util import img_as_ubyte
from tqdm import tqdm
from fpdf import FPDF, HTMLMixin
from datetime import datetime
import subprocess
from pip._internal.operations.freeze import freeze
from skimage import data
from skimage import filters

# ------- Variable specific to Cellpose -------

from urllib.parse import urlparse
%matplotlib inline
from cellpose import models
from skimage.util import img_as_ubyte
import cv2
from cellpose import plot
from ipywidgets import interact, interact_manual

from scipy.ndimage import gaussian_laplace

#For pyclesperanto

import pyclesperanto_prototype as cle

gpu_devices = cle.available_device_names(dev_type="gpu")
print("Available GPU OpenCL devices:" + str(gpu_devices))
cle.select_device(gpu_devices[0])
print("Using OpenCL device " + cle.get_device().name)

# Download the StarDist model

pretrained_model_name = "2D_versatile_fluo"
pretrained_model_path = "/content/"+pretrained_model_name

if os.path.exists(pretrained_model_path):
  shutil.rmtree(pretrained_model_path)
os.makedirs(pretrained_model_path)

wget.download("https://cloud.mpi-cbg.de/index.php/s/1k5Zcy7PpFWRb0Q/download?path=/versatile&files=2D_versatile_fluo.zip", pretrained_model_path)

with zipfile.ZipFile(pretrained_model_path+"/2D_versatile_fluo.zip", 'r') as zip_ref:
  zip_ref.extractall(pretrained_model_path)

h5_file_path = os.path.join(pretrained_model_path, "weights_best.h5")


# StarDist prediction function

def stardist_WF_prediction(timelapse, Saving_folder, StarDist_cancer_cells_WF_model_name, StarDist_cancer_cells_WF_model_path):

  # normalize channels independently
  axis_norm = (0,1)
  model_WF = StarDist2D(None, name = StarDist_cancer_cells_WF_model_name, basedir = StarDist_cancer_cells_WF_model_path)

  Number_of_nuclei_list = []
  Number_of_frame_list = []

  timelapse = normalize(timelapse, 1,99.8, axis=(0,)+tuple(1+np.array(axis_norm)))

  n_timepoint = timelapse.shape[0]
  prediction_stack = np.zeros((n_timepoint, timelapse.shape[1], timelapse.shape[2]), dtype=np.float32)

# Analyse each time points one after the other

  for t in range(n_timepoint):
    img_t = timelapse[t]
    labels, polygons = model_WF.predict_instances(img_t)
    prediction_stack[t] = labels
    Nuclei_array = polygons['coord']
    Nuclei_array2 = [str(t), Nuclei_array.shape[0]]
    Number_of_nuclei_list.append(Nuclei_array2)
    Number_of_frame_list.append(t)

  prediction_stack = img_as_float32(prediction_stack, force_copy=False)

# Export a csv file containing the number of nuclei detected at each frame
  my_df = pd.DataFrame(Number_of_nuclei_list)
  my_df.to_csv(Saving_folder_StarDist_cancer_cells+'/'+str(short_name[0])+'_Nuclei_number.csv', index=False, header=False)
  os.chdir(Saving_folder_StarDist_cancer_cells)
  imsave(str(short_name[0])+".tif", prediction_stack, compression ='zlib')


  # Object detected vs frame number
  plt.figure(figsize=(20,5))
  my_df.plot()
  plt.title('Number of objects vs frame number')
  plt.ylabel('Number of detected objects')
  plt.xlabel('Frame number')
  plt.legend()
  plt.savefig(Saving_folder_StarDist_cancer_cells+'/'+str(short_name[0])+'_Object_detected_vs_frame_number.png',bbox_inches='tight',pad_inches=0)
  plt.show()

  del prediction_stack
  del timelapse
  del model_WF
  del Number_of_nuclei_list
  del Number_of_frame_list
  del my_df


# Pix2pix prepare the data function

def pix2pix_prepare_data(timelapse):
  n_timepoint = timelapse.shape[0]
  sharpness_WF_sobel = []
  sharpness_WF_laplace = []

  for t in range(n_timepoint):
    img_t = timelapse[t]
    img_t = exposure.equalize_adapthist(img_t, clip_limit=0.03)
  # Calculate the gradient of the image in the horizontal and vertical directions using the Sobel operator
    grad_x = cv2.Sobel(img_t, cv2.CV_64F, 1, 0, ksize=5)
    grad_y = cv2.Sobel(img_t, cv2.CV_64F, 0, 1, ksize=5)

  # Calculate the magnitude of the gradient
    grad_magnitude = np.sqrt(np.square(grad_x) + np.square(grad_y))

  # Calculate the mean of the gradient magnitude
    sharpness_sobel = np.mean(grad_magnitude)
    sharpness_WF_sobel.append(sharpness_sobel)

    log_image = gaussian_laplace(img_t, sigma=1)

    # Calculate the variance of the LoG image
    sharpness_laplace = np.var(log_image)
    sharpness_WF_laplace.append(sharpness_laplace)

  threshold_sharpness_sobel = np.percentile(sharpness_WF_sobel, 75)
  threshold_sharpness_laplace = np.percentile(sharpness_WF_laplace, 75)


  for t in range(n_timepoint):
    img_t = timelapse[t]
    img_t = exposure.equalize_adapthist(img_t, clip_limit=0.03)

  # Calculate the gradient of the image in the horizontal and vertical directions using the Sobel operator
    grad_x = cv2.Sobel(img_t, cv2.CV_64F, 1, 0, ksize=5)
    grad_y = cv2.Sobel(img_t, cv2.CV_64F, 0, 1, ksize=5)

  # Calculate the magnitude of the gradient
    grad_magnitude = np.sqrt(np.square(grad_x) + np.square(grad_y))

    # Calculate the mean of the gradient magnitude
    sharpness_sobel = np.mean(grad_magnitude)

    log_image = gaussian_laplace(img_t, sigma=1)

    # Calculate the variance of the LoG image
    sharpness_laplace = np.var(log_image)

    if sharpness_sobel > threshold_sharpness_sobel and sharpness_laplace > threshold_sharpness_laplace:

      img_t = 255 * img_t # Now scale by 255
      img_t = img_t.astype(np.uint8)
      cv2.imwrite(testA_Folder+"/"+short_name[0]+"_"+str(t)+".png", img_t)
      cv2.imwrite(testB_Folder+"/"+short_name[0]+"_"+str(t)+".png", img_t)

# Here we create a merged A / A image for the prediction
  os.chdir("/content")
  !python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A "$imageA_folder" --fold_B "$imageB_folder" --fold_AB "$imageAB_folder"


def pix2pix_prepare_data_QC(timelapse):
  n_timepoint = timelapse.shape[0]
  for t in range(n_timepoint):
    img_t = timelapse[t]
    img_t = exposure.equalize_adapthist(img_t, clip_limit=0.03)
    img_t = 255 * img_t # Now scale by 255
    img_t = img_t.astype(np.uint8)
    cv2.imwrite(testA_Folder+"/"+short_name[0]+"_"+str(t)+".png", img_t)
    cv2.imwrite(testB_Folder+"/"+short_name[0]+"_"+str(t)+".png", img_t)
# Here we create a merged A / A image for the prediction
  os.chdir("/content")
  !python pytorch-CycleGAN-and-pix2pix/datasets/combine_A_and_B.py --fold_A "$imageA_folder" --fold_B "$imageB_folder" --fold_AB "$imageAB_folder"



# Pix2pix functions
def pix2pix_WF(imageAB_folder, Prediction_model_name, Prediction_model_path, Image_min_dim, patch_size, checkpoint, saving_folder):

  os.chdir("/content")
  #!python pytorch-CycleGAN-and-pix2pix/test.py --dataroot "$imageAB_folder" --name "$Prediction_model_name" --model pix2pix --no_dropout --preprocess scale_width --load_size $Image_min_dim --crop_size $Image_min_dim --results_dir "$saving_folder" --checkpoints_dir "$Prediction_model_path" --num_test $n_timepoint --epoch $checkpoint --input_nc "3" --output_nc "3"
  !python pytorch-CycleGAN-and-pix2pix/test.py --dataroot "$imageAB_folder" --name "$Prediction_model_name" --model pix2pix --no_dropout --preprocess scale_width --load_size $Image_min_dim --crop_size $Image_min_dim --results_dir "$saving_folder" --checkpoints_dir "$Prediction_model_path" --num_test $n_timepoint --epoch $checkpoint --input_nc "1" --output_nc "1" --dataset_mode "aligned"

  Checkpoint_name = "test_"+str(checkpoint)

  Prediction_results_folder = saving_folder+"/"+Prediction_model_name+"/"+Checkpoint_name+"/images"

  Prediction_results_images = os.listdir(Prediction_results_folder)

  for f in Prediction_results_images:
    if (f.endswith("_fake_B.png")):
      shutil.copyfile(Prediction_results_folder+"/"+f,saving_folder+"/"+f)

  shutil.rmtree(saving_folder+"/"+Prediction_model_name)

# Sum projection functions

def generate_sum_projections(input_folder,Saving_folder, name):
# Here we check the image dimensions
  random_choice = random.choice(os.listdir(input_folder))
  x = io.imread(input_folder+"/"+random_choice)
  Image_Y = x.shape[0]
  Image_X = x.shape[1]

#Here we load all the images as a stack to generate the projection
  n_images = len([name for name in os.listdir(input_folder)])
  image_stack = np.zeros((n_images, Image_Y, Image_X))
  Images_name = os.listdir(input_folder)
  for t in range(n_images):
    Image_t = io.imread(input_folder+"/"+Images_name[t], as_gray= True)
    image_stack[t] = Image_t

  sum_projection= np.sum(image_stack, axis=0)
  image_stack = 255 * image_stack # Now scale by 255
  image_stack = image_stack.astype(np.uint8)
  os.chdir(Saving_folder)
  imsave(str(short_name[0])+"_"+name+"_stack.tif", image_stack)
  imsave(str(short_name[0])+"_"+name+"_projection.tif", sum_projection)
  del image_stack

# Cellpose functions

def cellpose_PECAM(Input_folder, Result_folder):
  source_image_PECAM = Input_folder+"/"+str(short_name[0])+"_PECAM_projection.tif"
  source_image_PECAM = io.imread(source_image_PECAM)

  source_image_DAPI = Input_folder+"/"+str(short_name[0])+"_DAPI_projection.tif"
  source_image_DAPI = io.imread(source_image_DAPI)

  RGB_image = np.zeros((1024, 1024, 3))

  RGB_image[:, :, 0] = source_image_PECAM
  RGB_image[:, :, 1] = source_image_DAPI

  # Load CellPose parameters
  Object_diameter =  0
  Flow_threshold = 0.4
  Cell_probability_threshold=0
  segment_channel = 1
  nuclear_channel = 2

  channels=[segment_channel,nuclear_channel]
  model = models.Cellpose(gpu=True, model_type="cyto2")
  print("Cytoplasm2 model enabled")

  # CellPose detection
  masks, flows, styles, diams = model.eval(RGB_image, diameter=Object_diameter, flow_threshold=Flow_threshold,cellprob_threshold=Cell_probability_threshold, channels=channels)
  os.chdir(Result_folder)
  imsave(str(short_name[0])+"_HUVEC_mask.tif", masks, compression ='zlib')


#StarDist for HUVECs

def StarDist_endothelial_cells(Input_folder,Result_folder, Prediction_model_name, Prediction_model_path ):
  from stardist.models import StarDist2D

  source_image_DAPI = Input_folder+"/"+str(short_name[0])+"_DAPI_projection.tif"
  source_image_DAPI = imread(source_image_DAPI)

  np.random.seed(32)
  lbl_cmap = random_label_cmap()

  axis_norm = (0,1)   # normalize channels independently

  model_fluo = StarDist2D(None, name = Prediction_model_name, basedir = Prediction_model_path)

  img = normalize(source_image_DAPI, 1,99.8, axis = axis_norm)
  labels, polygons = model_fluo.predict_instances(img)

  os.chdir(Result_folder)
  imsave(str(short_name[0])+"_HUVEC_nuclei.tif", labels, compression ='zlib')

def detect_label_edges(labelimages, Result_folder):
  labelimages = io.imread(labelimages)
  edges = filters.sobel(labelimages)
  edges = exposure.equalize_adapthist(edges, clip_limit=0.03)
  edges = 255 * edges # Now scale by 255
  edges[edges>0]=1
  edges_label = labelimages*edges
  os.chdir(Result_folder)
  imsave(str(short_name[0])+"_HUVEC_junctions.tif", edges_label, compression ='zlib')


def detect_label_edges_Clij(labelimages, Result_folder):
  labelimages = io.imread(labelimages)
  Image_Y = labelimages.shape[0]
  Image_X = labelimages.shape[1]
  edge_image_destination = np.zeros((Image_Y, Image_X), dtype = np.uint16)
  edge_image_destination_2 = np.zeros((Image_Y, Image_X), dtype = np.uint16)
  edge_image_destination = cle.detect_label_edges(labelimages)
  edge_image_destination = cle.pull(edge_image_destination)
  edge_image_destination_2 = np.multiply(labelimages, edge_image_destination)
  os.chdir(Result_folder)
  imsave(str(short_name[0])+"_HUVEC_junctions.tif", edge_image_destination_2, compression ='zlib')


def colorize(im, color, clip_percentile=0.1):
    """
    Helper function to create an RGB image from a single-channel image using a
    specific color.
    """
    # Check that we do just have a 2D image
    if im.ndim > 2 and im.shape[2] != 1:
        raise ValueError('This function expects a single-channel image!')

    # Rescale the image according to how we want to display it
    im_scaled = im.astype(np.float32) - np.percentile(im, clip_percentile)
    im_scaled = im_scaled / np.percentile(im_scaled, 100 - clip_percentile)
    im_scaled = np.clip(im_scaled, 0, 1)

    # Need to make sure we have a channels dimension for the multiplication to work
    im_scaled = np.atleast_3d(im_scaled)

    # Reshape the color (here, we assume channels last)
    color = np.asarray(color).reshape((1, 1, -1))
    return im_scaled * color

def plotresults(Result_folder):
  lbl_cmap = random_label_cmap()

  random_choice = random.choice(os.listdir(testA_Folder))
  InputWF = io.imread(testA_Folder+"/"+random_choice, as_gray = True)

  #InputWF = io.imread(testA_Folder +"/"+short_name[0] +"_0.png", as_gray = True)
  StarDistCancer = io.imread(Saving_folder_StarDist_cancer_cells +"/"+short_name[0] +".tif")
  StarDistHUVEC = io.imread(Saving_folder +"/"+short_name[0] +"_HUVEC_nuclei.tif")
  Pix2Pix_PECAM = io.imread(Saving_folder +"/"+short_name[0] +"_DAPI_projection.tif", as_gray = True)
  Pix2Pix_PECAM = colorize(Pix2Pix_PECAM, (0, 1, 0), clip_percentile=0.1)
  Pix2Pix_DAPI = io.imread(Saving_folder +"/"+short_name[0] +"_PECAM_projection.tif", as_gray = True)
  Pix2Pix_DAPI = colorize(Pix2Pix_DAPI, (1, 0, 1), clip_percentile=1)
  Huvec_Mask = io.imread(Saving_folder +"/"+short_name[0] +"_HUVEC_mask.tif")
  Huvec_edges = io.imread(Saving_folder +"/"+short_name[0] +"_HUVEC_junctions.tif")
  im_composite = np.clip(Pix2Pix_DAPI + Pix2Pix_PECAM, 0, 1)

  f=plt.figure(figsize=(16,8))
  plt.subplot(2,3,1)
  plt.imshow(StarDistCancer[0], cmap=lbl_cmap, interpolation='nearest')
  plt.title('Cancer cells')
  plt.axis('off');
  plt.subplot(2,3,2)
  plt.imshow(InputWF, cmap="gray", interpolation='nearest')
  plt.title('Input')
  plt.axis('off');
  plt.subplot(2,3,3)
  plt.imshow(im_composite, interpolation='nearest')
  plt.title('Pix2Pix predictions')
  plt.axis('off');
  plt.subplot(2,3,4)
  plt.imshow(StarDistHUVEC, cmap=lbl_cmap, interpolation='nearest')
  plt.title('HUVEC nuclei')
  plt.axis('off');
  plt.subplot(2,3,5)
  plt.imshow(Huvec_Mask, cmap=lbl_cmap, interpolation='nearest')
  plt.title('HUVECs segmentation')
  plt.axis('off');
  plt.subplot(2,3,6)
  plt.imshow(Huvec_edges, cmap=lbl_cmap, interpolation='nearest')
  plt.title('HUVEC junctions')
  plt.axis('off');
  plt.savefig(Result_folder+'/'+str(short_name[0])+'_results.png',bbox_inches='tight',pad_inches=0)
  plt.show()

print('----------------------------')
print("Libraries installed")



# **2. Select your parameters and paths**
---

In [ ]:
latest = "latest"

#@markdown #Provide the path to your dataset:

Data_folder = "" #@param {type:"string"}
Results_folder = "" #@param {type:"string"}

#@markdown #StarDist models locations:

StarDist_model_cancer_cells_WF = "" #@param {type:"string"}
StarDist_model_endothelial_cells = "" #@param {type:"string"}


#@markdown #Pix2pix models locations:

#@markdown ###WF to Nuclei:
Pix2pix_model_Nuclei = "" #@param {type:"string"}

Pix2pix_model_Nuclei_checkpoint = None#@param {type:"raw"}

#@markdown ###WF to PECAM:

Pix2pix_model_PECAM = "" #@param {type:"string"}

Pix2pix_model_PECAM_checkpoint = None#@param {type:"raw"}

# Here we check that the StarDist models exist

StarDist_cancer_cells_WF_model_name = os.path.basename(StarDist_model_cancer_cells_WF)
StarDist_cancer_cells_WF_model_path = os.path.dirname(StarDist_model_cancer_cells_WF)

StarDist_cancer_cells_full_model_path = StarDist_cancer_cells_WF_model_path+'/'+StarDist_cancer_cells_WF_model_name+'/'

# Here we check that the pix2pix models exist
if os.path.exists(StarDist_cancer_cells_full_model_path):
  print("The "+StarDist_cancer_cells_WF_model_name+" network will be used to detect cancer cells.")
else:
  print(bcolors.WARNING+'!! WARNING: The chosen StarDist model for detecting the cancer cells does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')

StarDist_model_endothelial_cells_name = os.path.basename(StarDist_model_endothelial_cells)
StarDist_model_endothelial_cells_path = os.path.dirname(StarDist_model_endothelial_cells)

StarDist_model_endothelial_cells_full_model_path = StarDist_model_endothelial_cells_path+'/'+StarDist_model_endothelial_cells_name+'/'

# Here we check that the pix2pix models exist
if os.path.exists(StarDist_model_endothelial_cells_full_model_path):
  print("The "+StarDist_model_endothelial_cells_name+" network will be used to detect cancer cells.")
else:
  StarDist_model_endothelial_cells_name = "fluo"

#Here we find the loaded model name and parent path
Pix2pix_model_Nuclei_model_name = os.path.basename(Pix2pix_model_Nuclei)
Pix2pix_model_Nuclei_model_path = os.path.dirname(Pix2pix_model_Nuclei)

#here we check if the pix2pix model exists
Pix2pix_model_Nuclei_full_model_path = Pix2pix_model_Nuclei_model_path+'/'+Pix2pix_model_Nuclei_model_name+'/'

if os.path.exists(Pix2pix_model_Nuclei_full_model_path):
  print("The "+Pix2pix_model_Nuclei_model_name+" network will be used.")
else:
  W  = '\033[0m'  # white (normal)
  R  = '\033[31m' # red
  print(R+'!! WARNING: The chosen model does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')


#Here we find the PECAM loaded model name and parent path
Pix2pix_model_PECAM_model_name = os.path.basename(Pix2pix_model_PECAM)
Pix2pix_model_PECAM_model_path = os.path.dirname(Pix2pix_model_PECAM)

#here we check if the pix2pix model exists
Pix2pix_model_PECAM_full_model_path = Pix2pix_model_PECAM_model_path+'/'+Pix2pix_model_PECAM_model_name+'/'

if os.path.exists(Pix2pix_model_PECAM_full_model_path):
  print("The "+Pix2pix_model_PECAM_model_name+" network will be used.")
else:
  W  = '\033[0m'  # white (normal)
  R  = '\033[31m' # red
  print(R+'!! WARNING: The chosen model does not exist !!'+W)
  print('Please make sure you provide a valid model path and model name before proceeding further.')






# **3. Process your data**
---

In [ ]:
#@markdown #Let's go

#Here we perform the predictions

for image in os.listdir(Data_folder):

  print("Performing prediction on: "+image)

  timelapse = imread(Data_folder+"/"+image)

  short_name = os.path.splitext(image)

  Image_Y = timelapse.shape[1]
  Image_X = timelapse.shape[2]
  n_timepoint = timelapse.shape[0]

  Image_min_dim = 1024
  patch_size = 1024

  Saving_folder = Results_folder+"/"+short_name[0]

  if os.path.exists(Saving_folder):
    shutil.rmtree(Saving_folder)
  os.makedirs(Saving_folder)

  #StarDist predictions. Here we identify the cancer cells

  Saving_folder_StarDist_cancer_cells = Saving_folder+"/"+"Cancer_cells_StarDist"
  if os.path.exists(Saving_folder_StarDist_cancer_cells):
    shutil.rmtree(Saving_folder_StarDist_cancer_cells)
  os.makedirs(Saving_folder_StarDist_cancer_cells)

  stardist_WF_prediction(timelapse, Saving_folder, StarDist_cancer_cells_WF_model_name, StarDist_cancer_cells_WF_model_path)

  #Pix2pix here we predict the nuclei and the cell-cell junction

  Saving_path_pix2pix = "/content/pix2pix"
  if os.path.exists(Saving_path_pix2pix):
    shutil.rmtree(Saving_path_pix2pix)
  os.makedirs(Saving_path_pix2pix)

  imageA_folder = Saving_path_pix2pix+"/Raw/A"
  os.makedirs(imageA_folder)

  imageB_folder = Saving_path_pix2pix+"/Raw/B"
  os.makedirs(imageB_folder)

  imageAB_folder = Saving_path_pix2pix+"/Raw/AB"
  os.makedirs(imageAB_folder)

  testAB_Folder = Saving_path_pix2pix+"/Raw/AB/test"
  os.makedirs(testAB_Folder)

  testA_Folder = Saving_path_pix2pix+"/Raw/A/test"
  os.makedirs(testA_Folder)

  testB_Folder = Saving_path_pix2pix+"/Raw/B/test"
  os.makedirs(testB_Folder)

  Saving_path_pix2pix_WF_DAPI = Saving_path_pix2pix +"/WF_DAPI"
  os.makedirs(Saving_path_pix2pix_WF_DAPI)

  Saving_path_pix2pix_WF_PECAM = Saving_path_pix2pix +"/WF_PECAM"
  os.makedirs(Saving_path_pix2pix_WF_PECAM)

  #pix2pix prepare the data
  pix2pix_prepare_data(timelapse)
  #pix2pix_prepare_data_QC(timelapse)

  #pix2pix nuclei
  pix2pix_WF(imageAB_folder, Pix2pix_model_Nuclei_model_name, Pix2pix_model_Nuclei_model_path, Image_min_dim, patch_size, Pix2pix_model_Nuclei_checkpoint, Saving_path_pix2pix_WF_DAPI)

  #pix2pix PECAM
  pix2pix_WF(imageAB_folder, Pix2pix_model_PECAM_model_name, Pix2pix_model_PECAM_model_path, Image_min_dim, patch_size, Pix2pix_model_PECAM_checkpoint, Saving_path_pix2pix_WF_PECAM)

  # Create the SUM projection for DAPI
  DAPI = "DAPI"
  generate_sum_projections(Saving_path_pix2pix_WF_DAPI,Saving_folder, DAPI)

  # Create the SUM projection for PECAM
  PECAM = "PECAM"
  generate_sum_projections(Saving_path_pix2pix_WF_PECAM,Saving_folder, PECAM)

  # Detect the endothelial cell
  cellpose_PECAM(Saving_folder, Saving_folder)

  # Detect the endothelial cell nuclei
  StarDist_endothelial_cells(Saving_folder, Saving_folder, StarDist_model_endothelial_cells_name, StarDist_model_endothelial_cells_path)

  # Detect the endothelial cell junctions
  #detect_label_edges(Saving_folder +"/"+short_name[0] +"_HUVEC_mask.tif", Saving_folder)
  detect_label_edges_Clij(Saving_folder +"/"+short_name[0] +"_HUVEC_mask.tif", Saving_folder)

  # make a figure
  plotresults(Saving_folder)



